In [3]:
import numpy as np
import pyniggli
from opf_python.lat_type import lat_type
from opf_python.niggli_lat_id import niggli_id
from opf_python.test_basis import test_cell

In [7]:
def niggli_setting(A):
    """Finds the corrdinates in G6 for the lattice A. Where G6 is
    [A[:,0].A[:,0],A[:,1].A[:,1],A[:,2].A[:,2],
    2*A[:,1].A[:,2],2*A[:,2].A[:,0],2*A[:,0].A[:,1]]
    
    Args:
        A (numpy array): The lattice vectors as columns of a matrix.
        
    Returns:
        G (list): The coordinates on G6 space.
    """
    
    a = A[:,0]
    b = A[:,1]
    c = A[:,2]
    A = np.dot(a,a)
    B = np.dot(b,b)
    C = np.dot(c,c)
    xi = np.dot(b,c)
    eta = np.dot(a,c)
    zeta = np.dot(a,b)
    
    G = [A,B,C,xi,eta,zeta]
    return G

# The Idea

The idea here is to try and see if we can figure out a method to transform the symmetry preserving supercell from our basis to the users basis by using the niggli reduced cell. The workflow would be as follows:

1) identify which of the 44 niggli cells is in use (there are two triclinic cells included in those 44 which we'll disregard)

2) Find the Supercell in the niggli basis

3) Transform back to the users basis.

UC = N

S = NHCinv

# Face centered cubic

## First basis

Face centered cubic makes a good test case for the cubic cells, all cubic cells have a single unique niggli cell.

In [4]:
U = np.transpose([[0.5,0,0.5],[0.5,0.5,0],[0,0,1]])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

Using the brute force method I've now confirmed that this niggli cell has the same symmetry preserving HNFs as our original basis choice.

In [5]:
test_cell(U)

('Users niggli number: ', 1)


True

## Second FCC basis

Let's do a second basis just to be sure.

In [6]:
U = np.transpose([[1,1,2],[1,0,1],[2,1,1]])

In [7]:
test_cell(U)

('Users niggli number: ', 1)


True

## FCC rotated basis (10 degrees about the z-axis)

In [8]:
R = [[np.cos(10.*np.pi/180.),-np.sin(10.*np.pi/180.),0],
     [np.sin(10.*np.pi/180.),np.cos(10.*np.pi/180.),0],
    [0,0,1]]
U = np.dot(R,U)

In [9]:
test_cell(U)

('Users niggli number: ', 1)


True

# Simple Cubic

In [10]:
As = [[[1,0,0],[0,1,0],[0,0,1]], [[1,0,0],[1,1,0],[0,0,1]],
        [[1,0,1],[1,1,0],[0,0,1]],[[1,1,1],[0,1,1],[1,0,1]],[[1,0,2],[0,1,0],[0,1,1]]]
As = [np.transpose(i) for i in As]

In [11]:
for U in As:
    print(U,test_cell(U))

('Users niggli number: ', 3)
(array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 1, 0],
       [0, 1, 0],
       [1, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 0, 0],
       [0, 1, 1],
       [2, 0, 1]]), True)


# Body Centered Cubic

In [12]:
As = [[[-1,1,1],[1,-1,1],[1,1,-1]],[[1,0,0],[0,1,0],[0.5,0.5,0.5]],
        [[1,0,0],[0.5,0.5,0.5],[0,0,1]],[[0.5,-0.5,0.5],[0,1,0],[0,0,1]],
        [[0,0,2],[1,-1,1],[1,1,-1]],[[-1,1,1],[2,0,0],[0,2,0]],
        [[1,-1,3],[1,-1,1],[1,1,-1]]]
As = [np.transpose(i) for i in As]

In [14]:
for U in As:
    print(test_cell(U))

('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True


# Simple Tetragonal

Now we should check a second crystal choice. Simple tetragonal cell's are another where we already have the symmetry preserving HNFs for one of the two niggli cells. We'll have to restrict ourselves to the niggli setting with lattice norms of [A,A,C] where C > A for these tests. The following 4 lattices satisfy that condition.

In [16]:
a1 = np.array([1,0,0])
a2 = np.array([0,1,0])
a3 = np.array([0,0,2])

As = [[a1+a2,a2,a3+a1],[a1+a2+a3,a2+a3,a1+a2],[a1,a2+a1,a3+a3],[-a1,-a2+a3,a3]]
As = [np.transpose(i) for i in As]

In [17]:
for U in As:
    print(test_cell(U))

('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True


## Third Basis Choice (rotated by 15 degrees)

In [18]:
R = [[np.cos(15.*np.pi/180.),-np.sin(15.*np.pi/180.),0],
     [np.sin(15.*np.pi/180.),np.cos(15.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[2])
U = np.dot(R,U)

In [19]:
test_cell(U)

('Users niggli number: ', 11)


True

## Fourth Basis Choice (rotated by 27 degrees)

In [23]:
R = [[np.cos(27.*np.pi/180.),-np.sin(27.*np.pi/180.),0],
     [np.sin(27.*np.pi/180.),np.cos(27.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[3])
U = np.dot(R,U)

In [24]:
test_cell(U)

('Users niggli number: ', 11)


True

# Simple Orthorhombic

In [3]:
As = [[[1.00000000, 0.00000000, 0.00000000],
       [0.00000000, 1.73205080, 0.00000000],
       [0.00000000, 0.00000000, 3.26598640]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [1.50000000, -1.50000000, 0.00000000]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [2.00000000, -2.00000000, 0.00000000]]]
As = [np.transpose(i) for i in As]

In [4]:
for U in As:
    print(test_cell(U,eps=1E-8))

('Users niggli number: ', 32)
True
('Users niggli number: ', 32)
True
('Users niggli number: ', 32)
True


# Conclusion

The tests so far indicate that the transformation to the Niggli cell will work. However there are still 39 cases to check and verify in addition to a more detailed study even of these systems.

What I need to do is find a way to quickly identify the lattice type, this can be accomplished via niggli faster than the current method, then verify that the supercell's retain symmetry for a larger test case.

In other words I need a far more extensive testing suite that gets wrapped into a subroutine so that it will automatically generate supercells for multiple densities for a given cell and for each supercell verify that the cell has equal or greater symmetry than the parent. If ever this isn't the case then we'll need to have it stop and report the error, i.e., let us know that the method doesn't actually work.

In [1]:
from opf_python.niggli_lat_id import niggli_id
import numpy as np

In [4]:
A = np.transpose([[1,2,2],[2,1,2],[2,2,1]])
niggli_id(A)

('niggli', array([[ 0.,  1.,  2.],
       [-1., -1.,  1.],
       [ 1.,  0.,  2.]]))


('rhombohedral', 9, 3)

In [8]:
B = pyniggli.reduced_cell(A)
niggli_setting(B.niggli)

[2.0, 2.0, 9.0, 1.0, 1.0, 1.0]

In [15]:
temp = pyniggli.reduced_cell(np.dot([[1,0,0],[-1,1,0],[-1,-1,3]],B.niggli))
niggli_setting(temp.niggli)

[5.0, 5.0, 11.0, 1.0, 1.0, 2.0]

In [28]:
A = np.transpose([[-1.00000000, 0.00000000,-1.00000000],
                  [1.51184,0,-0.845178],
                  [-0.255922,-1.44338,0.92259]])
niggli_id(A)

('niggli', array([[-1.      ,  1.51184 , -0.255922],
       [ 0.      ,  0.      , -1.44338 ],
       [-1.      , -0.845178,  0.92259 ]]))


('rhombohedral', 24, 3)

In [26]:
temp = niggli_setting(A)
temp

[2.0,
 2.9999860372840002,
 2.9999926302250004,
 -1.83332939414,
 -0.66666499999999995,
 -0.66666200000000009]

In [27]:
2*abs(temp[3]+temp[4]+temp[5])

6.3333127882799998

In [1]:
from opf_python.trig import rhom_2

In [2]:
count = 0
for i in range(501):
    count += len(rhom_2(i))
count

867

In [3]:
import time

In [4]:
start = time.time()
rhom_2(100000)
end = time.time()
print(end-start)

10.1472918987


In [16]:
from opf_python.trig import rhom_9
count = 0
for i in range(201):
    count += len(rhom_9(i))
count

53842

In [29]:
import time
start = time.time()
rhom_9(100000)
end = time.time()
print(end-start)

KeyboardInterrupt: 

In [3]:
from opf_python.trig import rhom_24
count = 0
for i in range(501):
    if i%100==0:
        print(i)
    count += len(rhom_24(i))
count

0
100
200
300
400
500


111